# The Hangman Game

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

In [2]:
# a function allowing interactive play
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

In [3]:
interactive = False  # set True to play the game interactively

In [4]:
# play the interactive game
if interactive:
    hangman('whatever', human, 8, True)

## Load datasets
### Preprocessing NLTK's Brown corpus

In [5]:
# load and preprocess words for test and train
from nltk.corpus import brown
import numpy as np
import re 

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []
 
valid_words = []
words = [word.lower() for word in brown.words()]   # lowercase the words
words = set(words)    # remove duplicates
 
for word in words:
    non_alpha = re.sub("[a-z]", "", word).strip()  
    if len(non_alpha) == 0:         # save if only alphabets exist in the word        
        valid_words.append(word)

np.random.shuffle(valid_words)        

test_set = valid_words[0:1000]
training_set = valid_words[1000:]

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


In [6]:
# test block
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

In [7]:
# play hangman using random words from test set
interactive = True
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)
interactive = False

## Implementation
### 1. Random guesser

In [8]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [9]:
import string
import random

alphabets = list(string.ascii_lowercase)

def random_guesser(mask, guessed, **kwargs):
    valid_alphabets = []
    
    for alpha in alphabets:
        if not (alpha in guessed) and not (alpha in mask):
            valid_alphabets.append(alpha)
        
    next_guess = random.choices(valid_alphabets)[0]

    return next_guess

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = loeser
Number of mistakes made by the random guesser = 11

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.692


In [10]:
# test block
assert(result > 10 and result < 20)

### 2. Unigram guesser

In [11]:
from collections import Counter

unigram_counts = None

unigram_counts = Counter()

for training_word in training_set:
    for alpha in training_word:
        unigram_counts[alpha] += 1

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    '''
    Each position has the same probability to have a specific character, 
    so the product based on a specific position is not considered. 
    All probabilities share the same denominator, so frequency is used instead of probability. 
    These ways can simplify the solution.
    '''
    freq_dict = {} 
    
    for alpha in alphabets:
        if (not alpha in guessed) and (not alpha in mask):
            freq_dict[alpha] = unigram_counts[alpha]
            
    next_guess = max(freq_dict, key=freq_dict.get)
    
    return next_guess        
                        
result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.385


In [12]:
# test block
assert(result > 5 and result < 15)

### 3. Unigram guesser by length (conditional unigram)

In [13]:
from collections import defaultdict

unigram_counts_by_length = None

unigram_counts_by_length = defaultdict(Counter)

for training_word in training_set:
    for alpha in alphabets: 
        length = len(training_word)
        count = training_word.count(alpha)
        unigram_counts_by_length[length][alpha] += count

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):

    prob_dict = {}
    length = len(mask)
    
    for alpha in alphabets:
        if (alpha in guessed) or (alpha in mask):
            continue
        elif not length in unigram_counts_by_length.keys():
            prob_dict[alpha] = unigram_counts[alpha]
        else:
            prob_dict[alpha] = unigram_counts_by_length[length][alpha]
 
    next_guess = max(prob_dict, key=prob_dict.get)
    
    return next_guess  

result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.352


In [14]:
# test block
assert(result > 5 and result < 15)

### 4. Bigram guesser

In [15]:
bigram_counts = None

bigram_counts = defaultdict(Counter)

for training_word in training_set:
    word = '$' + training_word        # set a starting mark
    for i in range(0, len(word)-1):
        left_alpha = word[i]
        alpha = word[i+1]
        bigram_counts[left_alpha][alpha] += 1

total_unigram_counts = sum(unigram_counts.values())

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts = unigram_counts): # add extra arguments if needed
    prob_dict = defaultdict(lambda : 0)
    
    for alpha in alphabets:
        if (alpha in guessed) or (alpha in mask):
            continue
        for i in range(0,len(mask)): 
            if mask[i] != '_':          # continue if the position is guessed already 
                continue
            
            left_alpha = '$' if i == 0 else mask[i-1]   
                
            if left_alpha == '_':       # if no conditional alphabet exists on the left, use unigram counts result
                prob_dict[alpha] += unigram_counts[alpha] / total_unigram_counts
            else:
                if left_alpha in bigram_counts.keys():
                    total_bigram_counts = sum(bigram_counts[left_alpha].values())
                    prob_dict[alpha] += bigram_counts[left_alpha][alpha] / total_bigram_counts
                           
    next_guess = max(prob_dict, key=prob_dict.get)
    
    return next_guess

result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.729


### 5. My guesser

In [16]:
# initialise dictionaries bidirectional n-gram models
bigram_counts_rev = defaultdict(Counter)
trigram_counts = defaultdict(lambda: defaultdict(Counter))
trigram_counts_rev = defaultdict(lambda: defaultdict(Counter))

# set the optimal parameters
k = 0.01
v = len(alphabets)
uni_gamma, bi_gamma, tri_gamma = 0.1, 0.2, 0.7

# generate n-gram dictionaries
for training_word in training_set:
    word = '$' + training_word + '/'
    for i in range(1, len(word)-1):
        right1_alpha = word[i+1]
        alpha = word[i]
        bigram_counts_rev[right1_alpha][alpha] += 1
        if i >= 2:
            left1_alpha = word[i-1]
            left2_alpha = word[i-2]
            trigram_counts[left2_alpha][left1_alpha][alpha] += 1
        if i < (len(word)-2):
            right2_alpha = word[i+2]
            trigram_counts_rev[right2_alpha][right1_alpha][alpha] += 1

# count the vocab sizes
bi_v, bi_v_rev, tri_v, tri_v_rev = 0, 0, 0, 0
for left1_alpha in bigram_counts.keys():
    bi_v += len(bigram_counts[left1_alpha].keys())
for right1_alpha in bigram_counts_rev.keys():
    bi_v_rev += len(bigram_counts_rev[right1_alpha].keys())
for left1_alpha in trigram_counts.keys():
    for left2_alpha in trigram_counts[left1_alpha].keys():
        tri_v += len(trigram_counts[left2_alpha][left1_alpha].keys())
for right1_alpha in trigram_counts_rev.keys():
    for right2_alpha in trigram_counts_rev[right1_alpha].keys():
        tri_v_rev += len(trigram_counts_rev[right2_alpha][right1_alpha].keys())
        
def my_amazing_ai_guesser(mask, guessed, unigram_counts = unigram_counts, \
                          bigram_counts=bigram_counts, bigram_counts_rev = bigram_counts_rev, \
                          trigram_counts = trigram_counts, trigram_counts_rev = trigram_counts_rev ): 
    
    prob_dict = defaultdict(lambda:0)
    
    for alpha in alphabets:
        if (alpha in guessed) or (alpha in mask):
            prob_dict[alpha] = 0
            continue
    
        for i in range(0,len(mask)): 
            if mask[i] != '_':
                continue   
            
            prob_dict[alpha] += uni_gamma*((unigram_counts[alpha] +k)/ (total_unigram_counts + k*v))
    
            left1_alpha = '$' if i == 0 else mask[i-1]
            right1_alpha = '/' if i == len(mask)-1 else mask[i+1]
            
            if left1_alpha != '_': 
                total_bigram_counts = sum(bigram_counts[left1_alpha].values())
                prob_dict[alpha] += bi_gamma*((bigram_counts[left1_alpha][alpha]+ k)/ \
                                              (total_bigram_counts + k*bi_v))
            if right1_alpha != '_':
                total_bigram_counts_rev = sum(bigram_counts_rev[right1_alpha].values())
                prob_dict[alpha] += bi_gamma*((bigram_counts_rev[right1_alpha][alpha] +k)/ \
                                        (total_bigram_counts_rev + k*bi_v_rev))
            if i >= 1:
                left2_alpha = '$' if i == 1 else mask[i-2]
                if not '_' in [left1_alpha, left2_alpha]: 
                    total_trigram_counts = sum(trigram_counts[left2_alpha][left1_alpha].values())
                    prob_dict[alpha] += tri_gamma*((trigram_counts[left2_alpha][left1_alpha][alpha]+ k)/ \
                                            (total_trigram_counts + k*tri_v))
            if i <= (len(mask)-2):
                right2_alpha = '/' if i == len(mask)-2 else mask[i+2]
                if not '_' in [right1_alpha, right2_alpha]: 
                    total_trigram_counts_rev = sum(trigram_counts_rev[right2_alpha][right1_alpha].values())
                    prob_dict[alpha] += tri_gamma*((trigram_counts_rev[right2_alpha][right1_alpha][alpha]+ k)/ \
                                            (total_trigram_counts_rev + k*tri_v_rev))
                   
    next_guess = max(prob_dict, key=prob_dict.get)
    return next_guess

result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.658
